In [16]:
import chess
import numpy as np
from tqdm import tqdm
from engine.evaluate import evaluate_board
from engine.nn_eval import initialize_model
from engine.nn.utils.feature_extract import initialize_stockfish, get_position_score
from engine.nn.utils.random_board import generate_random_board

In [2]:
model = initialize_model('engine/nn/models/model_transformer.pth')
stockfish = initialize_stockfish('stockfish/stockfish-ubuntu-x86-64-avx2')

In [19]:
np.clip(1231, 0, 10)

10

In [20]:
def get_score(iterations: int, same_signal_buf=0.75, clip_min=-15, clip_max=15):
    abs_diff = []
    
    def have_same_sign(num1, num2, limit=0.2):
        if np.abs(num1) <= limit and np.abs(num2) <= limit:
            return True
        
        return (num1 * num2) > 0
    
    for _ in tqdm(range(iterations)):
        board = generate_random_board()
        my_eval = np.clip(evaluate_board(board, model), clip_min, clip_max)
        stockfish_eval = np.clip(get_position_score(stockfish, board), clip_min, clip_max)
        
        buf = have_same_sign(my_eval, stockfish_eval) * same_signal_buf
        
        dif = np.abs(stockfish_eval - my_eval) * buf
        
        abs_diff.append(dif)
        
    return np.array(abs_diff)

In [21]:
abs_diff = get_score(250)

100%|██████████| 250/250 [01:59<00:00,  2.09it/s]


In [22]:
abs_diff.mean()

2.1533320169075507